In [ ]:
# Load Necessary Files 

import pandas as pd
import numpy as np

In [ ]:
# Import Csvs

df = pd.read_csv('Webmd_final.csv')
df1 = pd.read_csv('Wellrx_final.csv')

In [ ]:
# Check Webmd csv

df.head()

In [ ]:
# Check Wellrx csv 

df1.head()

In [ ]:
# Wellrx name and price list that includes all drug names and alternatives 

part1 = df1.iloc[:,0:2]
part2 = df1.iloc[:,2:4]

new_columns = ["drug name", "price"]
part1.columns = new_columns
part2.columns = new_columns

vert_df1 = pd.concat([part1, part2], ignore_index=True)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(vert_df1)

In [ ]:
# Column to link Webmd names and Wellrx (WebMD)

link = pd.DataFrame(data= df['drug'].unique())
link['drug_meta'] = link[0].str.split().str[0].str.lower()

In [ ]:
# Column to link Webmd names and Wellrx (Wellrx)

vert_df1['drug_meta'] = vert_df1['drug name'].str.split().str[0].str.lower()

In [ ]:
# Merge link columns. Get it ready to link to original Webmd dataframe 

final = pd.merge(link, vert_df1, how = 'left',on='drug_meta')
final = final.drop_duplicates(inplace = False)

In [ ]:
# Drop repeated drug names (Some drugs are repeated in WebMD common list/ the brand and generic)

final1 = final.drop(df.index[[17,42,57,66, 85, 105]])

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(final1)

In [ ]:
# Replace cipro with cirpfloxacin hcl (WellRX and Webmd had different naming)

final1.loc[24,0] = 'Cipro oral' 
final1.loc[24,'drug_meta'] = 'ciprofloxacin' 
final1.loc[24,'drug name'] = 'CIPROFLOXACIN HCL' 
final1.loc[24,'price'] = 6.69

# Replace LEVAQUIN for levofloxacin (WellRX and Webmd had different naming)

final1.loc[50,0] = 'Levaquin oral' 
final1.loc[50,'drug_meta'] = 'levofloxacin-d5w' 
final1.loc[50,'drug name'] = 'LEVOFLOXACIN-D5W' 
final1.loc[50,'price'] = 8.22

In [ ]:
# Reset indexes 

final1 = final1.reset_index()

In [ ]:
# Delete extra index column
del final1['index']


In [ ]:
# Rename drug column

final1 = final1.rename(columns={0: "drug"})

In [ ]:
# Merge original Webmd with modified Well

Webmd_and_Wellrx = pd.merge(df, final1, on="drug")

In [ ]:
# Get rid of linking columns

del Webmd_and_Wellrx['drug_meta']
del Webmd_and_Wellrx['drug name']

In [ ]:
# Export to csv

Webmd_and_Wellrx.to_csv('Webmd_and_Wellrx.csv', index = False )

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(Webmd_and_Wellrx.groupby(['drug']).mean())